## Download and load data

In [1]:
# !pip install GitPython -qU

In [2]:
"""
from git import Repo

repo = Repo.clone_from("https://github.com/langchain-ai/langserve", "./langserve_repo")
branch = repo.head.reference
"""

'\nfrom git import Repo\n\nrepo = Repo.clone_from("https://github.com/langchain-ai/langserve", "./langserve_repo")\nbranch = repo.head.reference\n'

In [3]:
# repo

In [4]:
# from langchain.document_loaders import GitLoader 
from langchain.document_loaders import PyPDFLoader

In [5]:
# loader = GitLoader(repo_path="./langserve_repo/", branch=branch)

loaders = [
    PyPDFLoader("../data/Glasnik-638.pdf"),
    PyPDFLoader("../data/Glasnik-689.pdf"),
    PyPDFLoader("../data/masteri-upatstvo.pdf"),
    PyPDFLoader("../data/pravila-za-seminarska-rabota_ekts.pdf")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [6]:
#langserve_data = loader.load()

In [7]:
#len(langserve_data)

## Split data

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#splitter = RecursiveCharacterTextSplitter(
#    chunk_size=512,
#    chunk_overlap=100,
#)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)

#langserve_data = splitter.split_documents(langserve_data)
splits = text_splitter.split_documents(docs)

In [9]:
#len(langserve_data)

In [10]:
len(splits)

90

In [19]:
# hf_embedding_url = "https://l28vfh3hv125bzuu.us-east-1.aws.endpoints.huggingface.cloud"

## Get embeddings

In [11]:
#import getpass
#import os

#hf_token = getpass.getpass("Enter your Hugging Face API token: ")

# Hugging face is not free so for testing we try out Cohere

import getpass
import os

if not os.getenv("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API key: ")

Enter your Cohere API key:  ········


In [12]:
#from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

#embeddings_model = HuggingFaceInferenceAPIEmbeddings(
#    api_key=hf_token, api_url=hf_embedding_url
#)

from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)

In [13]:
# Test example
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

embedding1 = embeddings.embed_query(sentence1)
embedding2 = embeddings.embed_query(sentence2)
embedding3 = embeddings.embed_query(sentence3)

In [14]:
import numpy as np # to compare embeddings

In [15]:
np.dot(embedding1, embedding2)

0.8963837577327499

In [16]:
np.dot(embedding1, embedding3)

0.18742213262546123

In [17]:
np.dot(embedding2, embedding3)

0.1376780688827518

In [18]:
"""
embeddings = []

for i in range(0, len(langserve_data) - 1, 32):
    response = embeddings_model.embed_documents(
            [document.page_content for document in langserve_data[i : i + 32]]
        )
    embeddings.append(
        response
    )
"""

#len(embeddings)

'\nembeddings = []\n\nfor i in range(0, len(langserve_data) - 1, 32):\n    response = embeddings_model.embed_documents(\n            [document.page_content for document in langserve_data[i : i + 32]]\n        )\n    embeddings.append(\n        response\n    )\n'

In [20]:
#embeddings = [item for sub_list in embeddings for item in sub_list]

In [21]:
#len(embeddings)

In [22]:
#!pip install -qU faiss-cpu # we do this once

In [29]:
len(splits)

90

In [28]:
# test if properly split
splits[0].page_content

'Univerzitetski glasnik  Br.  638, 30 декември 2022  \n \n1 \n   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \nUniverzitetski glasnik  \n \n \nИздавач  \nУниверзитет  „Св. Кирил  и \nМетодиј“  - Скопје  \n \n \n \n \n \n \n \n \n \nУредник  \nна издавачката  дејност  на \nУКИМ  \nпроф . д-р Никола  Јанкуловски , \nректор  \n \n \nУредник  \nна Универзитетски  гласник  \nм-р Марија  Маневска , \nгенерален  секретар  \n \nЛектор  \nВесна Илиевска Цветановска  \n \nТехничко  уредување  \nЗоран  Кордоски  \nм-р Елизабета Додевска  \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nО Д Л У КА  \nЗА ПРАВИЛА НА СТУДИРАЊЕ  \nна втор циклус студии (постдипломски  \nстудии)  на Универзитетот „Св. Кирил и \nМетодиј“ во Скопје , Економски и нститут  – \nСкопје'

In [30]:
from langchain.vectorstores import FAISS

langserve_text_data = [document.page_content for document in splits]

In [31]:
emb = embeddings.embed_documents(langserve_text_data)

In [34]:
len(emb) # corresponds to the number of split documents

90

In [37]:
# ova e od tutorijalot

document_embedding_pairs = list(zip(langserve_text_data, emb))

vectorstore = FAISS.from_embeddings(
    document_embedding_pairs,
    embedding=embeddings,
)

In [38]:
vectorstore

In [40]:
vectorstore.save_local("../langserve_index")